In [1]:
%pip install minio

  Obtaining dependency information for minio from https://files.pythonhosted.org/packages/9a/ae/c20305102e32f078ec5e77a5010378318307a2de213f61be6c5facb56f0f/minio-7.1.16-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 378.4 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
MINIO_HOST = 'localhost'
MINIO_ACCESS_KEY = 'admin'
MINIO_SECRET_KEY = 'admin1234'

In [2]:
from minio import Minio

minio_client = Minio(
    f"{MINIO_HOST}:9000",
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)

In [3]:
minio_client.bucket_exists('user-pics')

False

In [4]:
if not minio_client.bucket_exists('user-pics'):
    minio_client.make_bucket('user-pics')

In [5]:
minio_client.bucket_exists('user-pics')

True

In [6]:
minio_client.list_buckets()

[Bucket('user-pics')]

---

In [7]:
import io

In [8]:
with open('cat.jpg', 'rb') as f:
    data = io.BytesIO(f.read())

result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024
)
print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


In [10]:
data.seek(0)

0

In [11]:
result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat_meta.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024,
    metadata={'creationPlace': 'HSE, Moscow'}
)
print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat_meta.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


In [12]:
from datetime import datetime, timedelta
from minio.retention import Retention
from minio.commonconfig import GOVERNANCE, Tags

In [13]:
data.seek(0)

0

In [14]:
date = datetime.utcnow().replace(
    hour=0, minute=0, second=0, microsecond=0,
) + timedelta(days=30)

tags = Tags(for_object=True)
tags['usergroup'] = 'teacher'

result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat_tags.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024,
    metadata={'creationPlace': 'HSE, Moscow'},
    tags=tags,
)

print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat_tags.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


---

In [15]:
import json

In [ ]:
http://localhost:9000/user-pics/cat.jpg

In [ ]:
minio_client.get_bucket_policy('user-pics')

In [17]:
# https://awspolicygen.s3.amazonaws.com/policygen.html

policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Action": [
        "s3:GetObject"
      ],
      "Effect": "Allow",
      "Resource": "arn:aws:s3:::user-pics/*",
      "Principal": {"AWS": "*"}
    }
  ]
}

minio_client.set_bucket_policy('user-pics', json.dumps(policy))

In [18]:
minio_client.get_bucket_policy('user-pics')

'{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":["*"]},"Action":["s3:GetObject"],"Resource":["arn:aws:s3:::user-pics/*"]}]}'

---

In [19]:
from minio.commonconfig import SnowballObject

In [ ]:
data.seek(0)
with open('dog.jpeg', 'rb') as f:
    data_dog = io.BytesIO(f.read())

minio_client.upload_snowball_objects(
    'user-pics',
    [
        SnowballObject('many_dog.jpeg', data=data_dog, length=-1),
        SnowballObject('many_cat.jpeg', data=data, length=-1)
    ],
)

---

In [28]:
for i in minio_client.list_objects('user-pics'):
    print(i.object_name)

cat.jpg
cat_meta.jpg
cat_tags.jpg


In [30]:
for i in minio_client.list_objects('user-pics'):
    minio_client.remove_object('user-pics', i.object_name)

In [31]:
for i in minio_client.list_objects('user-pics'):
    print(i.object_name)

---

In [32]:
minio_client.remove_bucket('user-pics')

https://min.io/docs/minio/linux/developers/python/API.html